In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import os

In [ ]:
data_dir = 'data/test'
batch_count = 50
img_width = 224
img_height = 224
batch_size = 16

def transform_batches(data_dir, data_gen_dir):
    X_transformed = []
    Y = []
    datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True)

    count = 0
    for X_batch, Y_batch in datagen.flow_from_directory(
            data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            class_mode='binary',
            shuffle=True,
            save_to_dir=data_gen_dir):
        print('Batch ', count, '/', batch_count)
        X_transformed.append(model.predict_on_batch(X_batch))
        Y.append(Y_batch)
        count += 1
        if len(Y) == batch_count:
            print('All done, gubna!')
            X_transformed = np.concatenate(X_transformed)
            Y = np.concatenate(Y)
            break
    
    return X_transformed, Y

def train_top_model(X_train, Y_train, X_test, Y_test, top_model_weights_path, epochs = 50, batch_size = 32):
    
    model = Sequential()
    model.add(Flatten(input_shape=X_train.shape[1:]))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, Y_train,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_test, Y_test))
    model.save_weights(top_model_weights_path)
    return model

In [ ]:
model = applications.VGG16(include_top=False, weights='imagenet')

X_test, Y_test = transform_batches('data/test', 'data/generated')

X_train, Y_train = transform_batches('data/train', 'data/generated')

np.save('data/X_test_transformed.npy',X_test)
np.save('data/Y_test_transformed.npy',Y_test)
np.save('data/X_train_transformed.npy',X_train)
np.save('data/Y_train_transformed.npy',Y_train)

top_model = train_top_model(X_train, Y_train, X_test, Y_test, 'data/top_model_1.h5')